In [6]:
using Pkg; Pkg.DEFAULT_IO[] = stdout; Pkg.activate("."); Pkg.instantiate();

  Activating environment at `~/Research/ToyModel.jl/Project.toml`


In [15]:
Pkg.status()

      Status `~/Research/ToyModel.jl/Project.toml`
  [6e4b80f9] BenchmarkTools v1.1.0
  [91a5bcdd] Plots v1.16.7


In [16]:
using BenchmarkTools

In [17]:
g = 9.81

Lx = 100
nx = 25
dx = Lx/nx
x = LinRange(0, Lx, nx+1)
x_trunc = x[1:nx];

In [22]:
function partial_x(f, dx)

    nx, = size(phi_true)
    dfdx = zeros(nx)
    for i in 2:nx
        dfdx[1] = 1/(dx) * (f[1] - f[nx])
        dfdx[i] = 1/(dx) * (f[i]-f[i-1])
    end
    return dfdx
end

function adv_x_Euler(f1, a, dx)
    nx, = size(phi_true)
    f2 = zeros(nx)
    f2 .= a .* partial_x(f1, dx)
    return f2
end;

In [28]:
#Initialising variables

phi_old = sin.(2*π*x/(Lx))
phi_old = phi_old[1:nx]
phi_new = zeros(nx)

a = 1 #Speed of advection

#Time variables
t_start = 0
t_end = 100
dt = 1

a = 1

#True solution
phi_true = sin.(2*π*x/(Lx))[1:nx]

#Needed for making animations
n_steps = Int(t_end/dt)
phi_data = zeros(nx, n_steps + 1)
phi_true_data = zeros(nx, n_steps + 1)

#Advecting the variable for fixed number of time steps


function stepforward_original(phi_old, t_start, t_end, dt)
    count = 1
    for t in t_start:dt:t_end
        phi_new = phi_old - dt*adv_x_Euler(phi_old, a, dx)
        @. phi_true = sin(2*π*(x_trunc - a*(t))/Lx)
        phi_old = phi_new
#         phi_data[:, count] = phi_old
#         phi_true_data[:, count] = phi_true
        count = count+1
    end
end

stepforward_original (generic function with 1 method)

In [32]:
@btime partial_x(phi_old, dx);

  4.987 μs (268 allocations: 4.86 KiB)


In [33]:
@btime adv_x_Euler(phi_old, a, dx);

  5.363 μs (273 allocations: 5.25 KiB)


In [34]:
@btime stepforward_original(phi_old, t_start, t_end, dt);

  789.414 μs (28785 allocations: 650.19 KiB)


Now let's try to optimize.

In [47]:
function partial_x!(dfdx, f, dx, nx)
    dfdx[1] = (f[1] - f[nx]) / dx
    for i in 2:nx
        dfdx[i] = (f[i] - f[i-1]) / dx
    end
end

function adv_x_Euler!(u∇φ, dfdx, f, a, dx, nx)
    partial_x!(dfdx, f, dx, nx)
    @. u∇φ = a * dfdx
end;

In [55]:
dφdx = similar(phi_old)
u∇φ = similar(phi_old)
φtrue = similar(phi_old);

In [49]:
@btime partial_x!(dφdx, phi_old, dx, nx);

  38.696 ns (0 allocations: 0 bytes)


In [50]:
@btime adv_x_Euler!(u∇φ, dφdx, phi_old, a, dx, nx);

  51.153 ns (0 allocations: 0 bytes)


In [73]:
function stepforward_optimized!(u∇φ, dφdx, phi_old, φtrue, t_start, t_end, dt, x_trunc, Lx, a)
    for t in t_start:dt:t_end
        adv_x_Euler!(u∇φ, dφdx, phi_old, a, dx, nx)
        @. phi_old -= dt * u∇φ
        @. φtrue = sin(2π * (x_trunc - a*t)/Lx)
#         phi_data[:, count] = phi_old
#         phi_true_data[:, count] = phi_true
    end
end

stepforward_optimized! (generic function with 1 method)

In [74]:
@btime stepforward_optimized!(u∇φ, dφdx, phi_old, φtrue, t_start, t_end, dt, x_trunc, Lx,  a)

  33.920 μs (0 allocations: 0 bytes)
